### Este notebook tem como função criar uma pasta chamada "Kaggle Faces Dataset" onde os rostos recortados dos vídeos serão guardados

Inicialmente, carregamos as dependências necessárias

In [5]:
import cv2

import pandas as pd

from facenet_pytorch import MTCNN

from PIL import Image
import glob, os

import torch
import torchvision
import numpy as np

import random

import time

import pathlib
from pathlib import Path

import numpy as np

from tqdm.notebook import tqdm

Define-se o device onde será rodada a detecção de rostos.

In [6]:
# Definimos um device onde os tensores estarão sendo processados
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}, {}'.format(device, torch.cuda.get_device_name()))

Running on device: cuda:0, GeForce RTX 2070


# 1° passo: Criar uma classe que cuida de todas as leituras dos vídeos

Cria-se uma classe que cuida da leitura de todos os vídeos do dataset.

In [7]:
# Cuida de lidar com o acesso aos vídeos e devolver os paths / labels corretamente
class Videos():
    def __init__(self, root="./Kaggle Dataset"):
        # Guarda o folder_path
        self.root = Path(root)
        
        folders = list(self.root.glob("*"))
        order = lambda x: int(x.stem.split("_")[-1])
        self.folders = sorted(folders, key=order)
        
    def getRandomVideo(self):
        # Lê o arquivo JSON que contém as informações dos deepfakes naquela pasta
        
        folder_path = random.choice(self.folders)
        
        metadata = pd.read_json(folder_path/'metadata.json').transpose() # Troca colunas e linhas para ficarEM no formato correto
        
        video_path = random.choice(list(folder_path)) # Devolve um WindowsPath
        
        video_name = video_path.name # videoaleatorio.mp4
        
        label = metadata.loc[video_name].label
        
        return video_path, video_name, label
    
    def getRandomLabeledVideo(self, label):
        if label not in ["FAKE", "REAL"]:
            print("Label deve ser FAKE ou REAL")
            return
        
        folder_path = random.choice(self.folders) # "./Kaggle Dataset/some_folder"
        
        metadata = pd.read_json(folder_path/'metadata.json').transpose() # Troca colunas e linhas para ficarEM no formato correto
        
        fake_videos = metadata[metadata.label == label].index # Filtra os videos que são FAKE

        video_path = folder_path/random.choice(fake_videos)
        
        video_name = video_path.name # videoaleatorio.mp4
        
        label = label
        
        return video_path, video_name, label
        
    def getEquivalentRealVideo(self, video_path):
        
        if not isinstance(video_path, pathlib.WindowsPath):
            video_path = Path(video_path)
            if not video_path.exists():
                print("Video path não existe.")
                return
            
        folder_path = video_path.parent
        
        metadata = pd.read_json(folder_path/'metadata.json').transpose() # Troca colunas e linhas para ficarEM no formato correto
        
        try: 
            video = metadata.loc[video_path.name]
        except:
            print("O vídeo indicado não existe no metadata.")
            return
        
        if video.label == "REAL":
            print("O vídeo em questão já é um vídeo real.")
            return
        
        real_video_name = video.original
        real_video_path = folder_path.joinpath(real_video_name)
        label = "REAL"
        
        return real_video_path, real_video_name, label
    
    def getAllVideosPath(self):
        for video_name, columns in self.metadata.iterrows():
            yield self.folder_path + '/' + video_name, video_name, columns[0] # Label
            
    def generateMetadataTestSplit(self, proportion=0.2): # 20% para o set de treinamento
        print("Ajuste de metadata.json em andamento...")
        np.random.seed(42)
        for folder in self.folders:
            metadata_path = folder/"metadata.json"
            metadata = pd.read_json(metadata_path).transpose()
            if "test" not in metadata.values:
                counts = metadata.label.value_counts()

                reals_quantity_to_test = int(proportion*counts["REAL"])
                fakes_quantity_to_test = int(proportion*counts["FAKE"])

                real_videos = metadata[metadata.label == 'REAL'].index
                fake_videos = metadata[metadata.label == 'FAKE'].index

                reals_to_test = np.random.choice(real_videos, size=reals_quantity_to_test)
                fakes_to_test = np.random.choice(fake_videos, size=fakes_quantity_to_test)

                metadata.loc[reals_to_test, 'split'] = "test"
                metadata.loc[fakes_to_test, 'split'] = "test"
            else:
                print("Pass...")
            metadata.transpose().to_json(metadata_path)
        print("Finalizado com sucesso!")

Função **showVideo()**: Mostra o vídeo para checagem.

In [8]:
def showVideo(video_path, resize_factor=0.6):
    
    # Captura o vídeo no path
    cap = cv2.VideoCapture(str(video_path))

    while(cap.isOpened()):
        ret, frame = cap.read() # Lê o próximo frame
        if ret: # Sucesso na leitura
            frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
            cv2.imshow('frame', frame)
            
            # Apertar a tecla 'q' para sair do vídeo.
            key = cv2.waitKey(25)
            if key == 113:
                break
                
        else:
            break

    cv2.destroyAllWindows()
    cap.release()

In [9]:
def showVideoWithDetection(video_path, label=None, padding=0, size=-1, separate_face_box=False, resize_factor=0.6):
    
    # Captura o vídeo no path
    cap = cv2.VideoCapture(str(video_path))
    
    # Configura a cor a ser colocada na LABEL
    if label == 'REAL':
        color = (0, 255, 0) # Verde
    elif label == "FAKE":
        color = (0, 0, 255) # Vermelho  
    else:
        color = (0, 255, 255)
    
    face = None
    
    while(cap.isOpened()):
        ret, frame = cap.read() # Lê o próximo frame
        if ret: # Sucesso na leitura
            boxes, _ = mtcnn.detect(Image.fromarray(frame)) # Detecta as imagens. O método detect só aceita numpy arrays
            if boxes is not None: # Só entra se rostos forem detectados
                for box in boxes: # Para cada uma das bouding boxes encontradas em um único frame (a princípio só deve ter uma)
                    box = [int(b) for b in box]
                    if separate_face_box:
                        face = frame[int(box[1] - padding):int(box[3] + padding), int(box[0] - padding):int(box[2] + padding)].copy()
                        if face is not None:
                            if size > 0:
                                face = cv2.resize(face, (size, size))
                            cv2.imshow('face', face)
                    cv2.putText(img=frame, text=label, org=(box[0], box[1]), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=color, thickness=2)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color=[0, 255, 0], thickness=5)

            frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
            cv2.imshow('frame', frame)
            
            # Apertar a tecla 'q' para sair do vídeo.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        else:
            break
    
    cv2.destroyAllWindows()
    cap.release()

# 2° passo: Observar alguns vídeos e ajustar o threshold do MTCNN para a detecção de rostos

**MTCNN** - *Multi-task Cascaded Convolutional Network*

Ajustamos os thresholds e utilizamos a função de mostrar os vídeos, verificando visualmente se ele se comporta bem na maioria dos casos.

In [10]:
# Margin não faz diferença se o método .detect() for utilizado
IMAGE_SIZE = 224
MARGIN = 0
MIN_FACE_SIZE = 90
THRESHOLDS = [0.68, 0.75, 0.80]
POST_PROCESS = False
SELECT_LARGEST = True
KEEP_ALL = False
DEVICE = device

# ----------------------------------

mtcnn = MTCNN(image_size=IMAGE_SIZE,
              margin=MARGIN, 
              min_face_size=MIN_FACE_SIZE, 
              thresholds=THRESHOLDS,
              post_process=POST_PROCESS,
              select_largest=SELECT_LARGEST, 
              keep_all=KEEP_ALL, 
              device=device)

### Objeto Videos

Definimos nosso objeto vídeos com o diretório onde contém as pastas do conjunto de dados.

In [11]:
# Instaciamos uma objeto da classe Videos
videos = Videos(root="./Kaggle Dataset")

#### Vídeo Real Aleatório

Para cancelar a visualização do vídeo em tempo real, apertar a tecla `q` do teclado.Para cancelar a visualização do vídeo em tempo real, apertar a tecla `q` do teclado.

In [12]:
video_path, video_name, label = videos.getRandomLabeledVideo(label="REAL")

# Podemos passar um padding para verificar a quantidade desejada de recorte ao redo do rosto detectado
showVideo(video_path)
showVideoWithDetection(video_path, label=label)

#### Vídeo Falso Aleatório

Para cancelar a visualização do vídeo em tempo real, apertar a tecla `q` do teclado.

In [13]:
video_path, video_name, label = videos.getRandomLabeledVideo(label="FAKE") # Recupera o caminho de um vídeo aleatório na pasta que esteja com a label 'FAKE'

showVideo(video_path)
showVideoWithDetection(video_path, label=label)

#### Vídeo Falso Aleatório e sua versão REAL

Para cancelar a visualização do vídeo em tempo real, apertar a tecla `q` do teclado.

In [14]:
video_path, video_name, label = videos.getRandomLabeledVideo(label="FAKE") # Recupera o caminho de um vídeo aleatório na pasta que esteja com a label 'FAKE'
video_path_real, video_name_real, label_real = videos.getEquivalentRealVideo(video_path)

showVideo(video_path)
showVideo(video_path_real)

Tudo estando ok, agora podemos separar todos os vídeos em um conjunto de treinamento para realizar a validação cruzada (80%) e um conjunto de vídeos finais para teste (20%).

In [16]:
#videos.generateMetadataTestSplit(proportion=0.2)

# 3° Passo: Criar uma classe que recebe um vídeo, detecta todos os rostos, rescorta eles e os salva no diretório em questão

Classe **CropSaver()**: Recorta todos os rostos de um vídeo dada uma taxa de checagem por frame.

In [37]:
class CropSaver(Videos):
    
    def __init__(self, root="./Kaggle Dataset", device="cuda"):
        super().__init__(root)
        # Instaciamos a MTCNN de forma interna
        IMAGE_SIZE = 224
        MARGIN = 0
        MIN_FACE_SIZE = 90
        THRESHOLDS = [0.68, 0.75, 0.80]
        POST_PROCESS = False
        SELECT_LARGEST = True
        KEEP_ALL = False
        DEVICE = device

        # ----------------------------------

        self.mtcnn = MTCNN(image_size=IMAGE_SIZE,
                      margin=MARGIN, 
                      min_face_size=MIN_FACE_SIZE, 
                      thresholds=THRESHOLDS,
                      post_process=POST_PROCESS,
                      select_largest=SELECT_LARGEST, 
                      keep_all=KEEP_ALL, 
                      device=DEVICE).eval()

    def callCropFaces(self, path_to_save="./Kaggle Faces Dataset", call="test-video", 
                      check_every_frame=30, batch_size=40, padding=0, size=-1):
        path_to_save = Path(path_to_save)
        if call == "test-video":
            video_path = list(self.folders[0].glob("*.mp4"))[0]
            metadata = pd.read_json(self.folders[0]/"metadata.json").transpose()
            video_name = video_path.name
            label = metadata.loc[video_name].label
            split = metadata.loc[video_name].split
            self.saveCropFaces(video_path, path_to_save, split, label, batch_size=batch_size,
                               padding=padding, size=size, check_every_frame=check_every_frame)
            print("All Good! Go check it!")
            
        elif call == "test-folder":
            videos_path = list(self.folders[0].glob("*.mp4"))
            metadata = pd.read_json(self.folders[0]/"metadata.json").transpose()
            print("-"*20 + f"Beggining folder {self.folders[0].parts[-1]}..." + "-"*20 )
            for video_path in tqdm(videos_path):
                video_name = video_path.name
                label = metadata.loc[video_name].label
                split = metadata.loc[video_name].split
                self.saveCropFaces(video_path, path_to_save, split, label, batch_size=batch_size,
                                   padding=padding, size=size, check_every_frame=check_every_frame)
                
        elif call == "all":
            for folder in tqdm(self.folders):
                videos_path = list(folder.glob("*.mp4"))
                metadata = pd.read_json(folder/"metadata.json").transpose()
                print("-"*20 + f"Beggining folder {folder.parts[-1]}..." + "-"*20 )
                for video_path in tqdm(videos_path):
                    video_name = video_path.name
                    label = metadata.loc[video_name].label
                    split = metadata.loc[video_name].split
                    self.saveCropFaces(video_path, path_to_save, split, label, batch_size=batch_size,
                                       padding=padding, size=size, check_every_frame=check_every_frame)

    def saveCropFaces(self, video_path, path_to_save, split, label, batch_size=40, padding=0, size=-1, check_every_frame=30):
        
        # Cria um path para salvar os rostos recortados do vídeo
        class_path = path_to_save.joinpath(split, label) # Path(./Kaggle Faces Dataset/train/FAKE ou REAL)
        
        # Instancia um VideoCapture do arquivo presente em video_path (no caso, o vídeo)
        cap = cv2.VideoCapture(str(video_path))
        
        # Pega, em inteiros, a quantidade de frames do vídeo
        v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if not class_path.exists():
            print(f"The folder {class_path} não é alcançável.")
            return
        
        # Inicializa frames como uma lista vazia
        frames = []
        face_count = 0
        
        # Entra num loop que percorre o vídeo até ele acabar
        for _ in range(1, v_len + 1):
            # Realiza um grab() no próximo frame, mas não o decodifica. Isso ajuda a agilizar o processo se não for necessário
            # recuperar todos os frames a todo o momento.
            success = cap.grab()
            # Só recorta o rosto se o frame atual for mod check_every_frame, ou seja, ele só decodifica de check_every_frame em check_every_frame frames.
            if not success:
                continue
            if _ == 1 or _ % check_every_frame == 0:
                success, frame = cap.retrieve()
            else:
                continue
            if not success:
                continue
            # Realiza um append do frame atual na lista frames (ele é capturado no formato BGR porém a MTCNN espera no formato RGB)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            frames.append(frame)
            
            # Se o tamanho dos frames alcançou o batch_size OU o meu iterador de frames chegou no máixmo juntamento com meu tamanho de frames dentro da lista sendo maior que zero, continua pra capturar o rosto
            if len(frames) >= batch_size or (_ == v_len and len(frames) > 0):
                # Utiliza o MTCNN para detectar todas as bounding boxes de todos os rostos
                boxes, probs = self.mtcnn.detect(frames)
                # Verifica se não foi obtida nenhuma bounding box em todo o batch
                if not all(x is None for x in boxes):
                    # Acessa cada um dos frames no batch
                    for i, boxes_f in enumerate(boxes):
                        # Verifica houve None para o frame atual
                        if boxes_f is not None:
                            # Acessa cada uma das bounding boxes dentro de um único frame (pode haver vários rostos)
                            for bbox in boxes_f:
                                # Obtém o rosto
                                face = frames[i].crop(box=(bbox[0]-padding, 
                                                           bbox[1]-padding, 
                                                           bbox[2]+padding, 
                                                           bbox[3]+padding))

                                # Se desejado, aplica um resize
                                if size > 0:
                                    face = face.resize((size, size))
                                    
                                # face_count serve para não ocorrer sobrescrição de mais de um rosto por frame
                                face_count += 1
                                
                                # Cria o path para o rosto atual
                                
                                path = class_path.joinpath(f"{video_path.parts[-2]} {video_path.parts[-1]} {face_count} .jpg")
                                # Salva o rosto na pasta correta.
                                face.save(path)

                # Após o batch ser aplicado, resetamos a lista
                frames = []

        # Solta o objeto do VideoCapture
        cap.release()

Instaciamos o objeto.

In [38]:
cropsaver = CropSaver(device=device)

Testamos agora para apenas 1 vídeo.

In [44]:
cropsaver.callCropFaces(call="test-video", check_every_frame=30)

All Good! Go check it!


Agora para um diretório inteiro.

In [ ]:
cropsaver.callCropFaces(call="test-folder", check_every_frame=15)

# 4° Passo: Utilizar o objeto criado e iterar em todo o dataset

E iniciamos o loop que pode levar de algumas horas até alguns dias para terminar, dependendo do hardware. Valores importante que devem ser setados nessa etapa por fim:
- `check_every_frame`: Define de quantos em quantos frames será realizada a inferência da rede para obter o rosto. Caso o valor seja 1 a rede realizará a inferência de 1 em 1 frame, ou seja, tentará encontrar e recortar os rostos de todos os frames de todos os vídeos. Esse processo pode ser extremamente custoso. Como são aproximadamente 100 mil vídeos de aproximadamente 10 segundos cada, a checagem de 1 em 1 frame resultaria em aproximadamente 30 milhões de imagens (considerando a taxa dos vídeos de 30 frames por segunda). Essa quantidade de imagens é mais de 20 vezes o dataset do ImageNet. Aqui a sugestão está deixar esse valor como 30 (de 1 em 1 segundo) ou 15 (de meio em meio segundo).

- `batch_size`: Define a quantidade máxima de imagens concatenadas que deve ser armazenada antes de realizar a inferência da MTCNN pela placa de vídeo. Para um hardware de 8 GB de memória de vídeo, o tamanho máximo de imagens está em torno de 80. Aqui o valor está setado pela metade uma vez que a quantidade máxima esperada de imagens concatenadas é aproximadamente 20 imagens, deixando uma margem.

- `padding`: Define a área extra que deve ser extraída considerada na hora de extrair um rosto. No caso foi considerado como 0.

- `size`: Define um tamanho padrão de saída para as imagens. Como pode existir a necessidade de se alterar esse tamanho como desejar, o valor não é fornecido, extraindo a imagem em seu tamanho original.

In [52]:
cropsaver.callCropFaces(call="all", check_every_frame=30)

  0%|          | 0/50 [00:00<?, ?it/s]

--------------------Beggining folder dfdc_train_part_0...--------------------


  0%|          | 0/1334 [00:00<?, ?it/s]

--------------------Beggining folder dfdc_train_part_1...--------------------


  0%|          | 0/1699 [00:00<?, ?it/s]

...

  0%|          | 0/2406 [00:00<?, ?it/s]

--------------------Beggining folder dfdc_train_part_48...--------------------


  0%|          | 0/2463 [00:00<?, ?it/s]

--------------------Beggining folder dfdc_train_part_49...--------------------


  0%|          | 0/3134 [00:00<?, ?it/s]